In [ ]:
%pip install chromadb
%pip install sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/lab5_nlp/train_1.csv', header=None, names = ['topic','title', 'text'])
df['ID'] = df.index

In [ ]:
df

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def cut_text(text):

  len_text = len(text)

  all_len = 0
  text_fragments = []
  overlay = 100

  while all_len+overlay<len_text:
    text_fragments.append(text[all_len:all_len+200])
    all_len+=overlay

  return text_fragments

def cut_text_by_sent(text):
  sentences = sent_tokenize(text)
  print(sentences)
  len_text = len(text)

  fragment_len = 0
  text_fragments = []
  text_fragment = []


  for sent in sentences:

    if len(sent)>200:
      text_fragments.append(sent)
    else:
      if fragment_len<200:
        text_fragment.append(sent)

        fragment_len+=len(sent)
      else:
        text_fragments.append(' '.join(text_fragment))
        text_fragment = [sent]
        fragment_len = 0

  text_fragments.append(' '.join(text_fragment))

  text_fragments_len_lim = []
  for fragment in text_fragments:
    if len(fragment)>200:
      frags = cut_text(fragment)
      for f in frags:
        text_fragments_len_lim.append(f)
    else:
        text_fragments_len_lim.append(fragment)
  return text_fragments_len_lim

In [ ]:
text = 'Необходимо записать ваш датасет в векторную базу данных и выполнить эксперименты по поиску схожих фрагментов текста, соответствующих запросу. Разбиение текстовых документов на фрагменты. Разработать алгоритм разбиения текстовых документов на фрагменты текста. Можно использовать уже существующие механизмы, например, разбиение по длине фрагмента текста в символах и пересечению с соседними фрагментами. Дополнительные баллы за усложненные варианты, например: учитывать границы токенов и предложений.Подготовка метаданных для каждого фрагмента, таких как класс документа, автор документа, ключевые слова и др. Векторизация фрагментов текста. В качестве метода векторизации можно использовать стороний api (huggingface, openai, etc.), w2v или любую другую модель на выбор. Рекомендуется применить модель с huggingface. Подходящие модели с huggingface можно выбрать по ссылке. Примеры моделей из huggingface:ai-forever/sbert_large_mt_nlu_ru (Russian language)sentence-transformers/paraphrase-multilingual-mpnet-base-v2 (50 languages)Создание Векторной Базы Данных (ВБД). Необходимо выбрать одну из доступных ВБД, например: Chroma (рекомендуемая с точки зрения простоты), Pinecone и т.д. Реализовать механизм загрузки и сохранения текстовых данных в ВБД. Поиск схожих фрагментов текста. Выбрать алгоритмы similarity для поиска схожих фрагментов текста. Реализовать механизм поиска схожих фрагментов по заданным запросам механизм поиска схожих фрагментов по заданным запросам механизм поиска схожих фрагментов по заданным запросам механизм поиска схожих фрагментов по заданным запросам механизм поиска схожих фрагментов по заданным запросам.'
text_fragments = cut_text_by_sent(text)

In [ ]:
text_fragments

In [ ]:
[len(f) for f in text_fragments]

In [ ]:
class Loader:
    def load_single_document(self, file_path: str):
      pass

    def load_documents(self, source_dir: str):
      pass

class Embedder():
  def __init__(self):
    self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

  def get_embeddings(self, sentences):
    return [[float(e) for e in list(emb)] for emb in list(self.model.encode(sentences))]



class ChromaDB():
    def __init__(self):
      # self.client = chromadb.Client()
      self.client = chromadb.PersistentClient(path="/content/gdrive/MyDrive/lab5_nlp")


    def clear(self, name):
      self.client.delete_collection(name=name)
      return self.client.list_collections()

    def get_collection(self, name):
      return self.client.get_collection(name=name)

    def get_collections(self):
      return self.client.list_collections()


class ChromaCollection():
    def __init__(self, collection_name, similarity, client):
      self.collection_name = collection_name
      self.similarity = similarity
      self.client = client
      self.collection = self.client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": similarity})
      # self.collection = self.client.get_collection(collection_name)

    def add(self, embeddings, texts, topics, ids):
      self.collection.add(
          embeddings = embeddings,
          documents = texts,
          metadatas = [{"source": "df", "topic":f"{topic}"} for i, topic in enumerate(topics)],
          ids = [f'id {i}' for i in ids]
)

    def query(self, embeddings, n_results):
      return self.collection.query(
          query_embeddings=embeddings,
          n_results=n_results,
          # where={"metadata_field": "is_equal_to_this"},
          # where_document={"$contains":"search_string"}
      )


    def get(self):
      return self.collection.get()

    def count(self):
      return self.collection.count()




In [ ]:
embedder = Embedder()

In [ ]:
# embeds = embedder.get_embeddings(df['text'][:40000])

In [ ]:
client = ChromaDB()

In [ ]:
client.get_collections()

In [ ]:
# client.clear('l2_sim')

In [ ]:
collection_cos = ChromaCollection('cos_sim', 'cosine', client.client)
collection_l2 = ChromaCollection('l2_sim', 'l2', client.client)
collection_Ip = ChromaCollection('Ip_sim', 'ip', client.client)

In [ ]:
# collection_cos.add(embeds[0:40000], df['text'].values.tolist()[0:40000], df['topic'].values.tolist()[0:40000], df['ID'].values.tolist()[0:40000])
# collection_l2.add(embeds[0:40000], df['text'].values.tolist()[0:40000], df['topic'].values.tolist()[0:40000], df['ID'].values.tolist()[0:40000])
# collection_Ip.add(embeds[0:40000], df['text'].values.tolist()[0:40000], df['topic'].values.tolist()[0:40000], df['ID'].values.tolist()[0:40000])

In [ ]:
# collection_cos.get()

In [ ]:
print(collection_cos.count())
print(collection_l2.count())
print(collection_Ip.count())

In [ ]:
questions = ['What are short-sellers seeing again?',
             'What is Carlyle Group known for?',
             'What factors are expected to hang over the stock market next week?',
             'Why have authorities halted oil export flows from the main pipeline in southern Iraq?',
             'What economic menace do world oil prices present?',
             'What offset the positive outlook from computer maker Dell Inc. on stocks?',
             'How much did the assets of the nation\'s retail money market mutual funds fall in the latest week?',
             'What indicators suggest that the economy is improving?',
             'What was Danny Bazil Riley\'s initial job and salary?',
             'What are short-sellers seeing again on Wall Street?',

             'Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums?',
             'Can OPEC do anything to douse oil prices in an oversupplied market?',
             'Non-OPEC oil exporters should consider increasing output to cool record crude prices, according to OPEC President Purnomo Yusgiantoro. What is the suggestion amidst soaring oil prices?',
             'Did the auction for Google Inc.\'s initial public offering start off smoothly?',
             'What happened to the dollar on Friday amidst data showing a record U.S. trade deficit?',
             'When should you have the money talk with your elderly relatives?',
             'Why has the back-to-school season become a huge marketing phenomenon?',
             'Who has lost less in the stock market compared to growth-focused investors?',
             'What has driven the US trade deficit to a record high?',
             'According to a press report, which company could be bracing itself for a possible takeover attempt?',

             'Were there last-minute worries over an interview with Google\'s bosses in Playboy magazine?',
             'According to official figures, is the eurozone economy still growing?',
             'What is causing the slowdown in economic growth in Japan?',
             'What was the impact of the South African central bank trimming interest rates without prior warning?',
             'According to a new survey, what happened to the cost of buying cars over the past five years?',
             'Why did South Korea\'s central bank cut interest rates?',
             'When did the auction of shares in Google take place?',
             'Why did Hewlett-Packard shares fall and what is their warning for the final quarter?',
             'What happened to one of the oldest textile operators on the Indian Ocean island of Mauritius?',
             'Why did Chad ask the IMF for a loan?',

             'Why is the company running the Japanese nuclear plant closing its reactors?',
             'What is Trevor Baylis planning to do with his company?',
             'Why is Saudi Arabia ready to push extra oil into the market?',
             'What are the plans of the group led by UAE\'s Etisalat after winning mobile phone licenses in Saudi Arabia?',
             'Why did Network Rail bring in specialist Indian engineers?',
             'What has been the focus of the scientists at NitroMed Inc. lately?',
             'What bid did the person submit in the auction-style initial public offering of Google Inc.?',
             'What is the reason given by Massachusetts bargain hunters for their increased shopping on the sales tax holiday?',
             'What conclusion did IBM Corp. researchers come to regarding e-mail?',
             'According to the article, in what ways did Bill Gates mess up as he guided Microsoft to greatness?',

             'What does columnist Steve Bailey argue for in relation to legal weapons?',
             'What contrasting views do President Bush and Senator John F. Kerry have on the US economy?',
             'Who is the Marlborough-based technology company suing and what are they accused of?',
             'What improvements are planned for Central Square in Lynn?',
             'What improvements are planned for Central Square in Lynn?',
             'What recent lawsuit refocuses attention on Gary Winnick?',
             'What is the reason behind Russia\'s emergence as a superpower?',
             'What does Kevin B. Rollins discuss in relation to Dell?',
             'What unique solution does Mr. Stein propose for the country\'s need for cash?',
             'Why did Quality Distribution experience a large loss in the second quarter?'

]

In [ ]:
q_embeds = embedder.get_embeddings(questions)

In [ ]:
questions = ['What offset the positive outlook from computer maker Dell Inc. on stocks?']


In [ ]:
q_embeds = embedder.get_embeddings(questions)

### cos

In [ ]:
results = collection_cos.query(q_embeds,10000)

In [ ]:
results['documents'][0][:10]

In [ ]:
ind= []
for i, res in enumerate(results['ids']):
  try:
    ind.append(res.index(f'id {i}'))
  except:
    ind.append(10000)

In [ ]:
ind

In [ ]:
sum(ind) / len(ind)

### l2

In [ ]:
results = collection_l2.query(q_embeds,1000)

In [ ]:
ind = []
for i, res in enumerate(results['ids']):
  try:
    ind.append(res.index(f'id {i}'))
  except:
    ind.append(1000)

In [ ]:
ind

In [ ]:
sum(ind) / len(ind)

### Ip

In [ ]:
results = collection_Ip.query(q_embeds,1000)

In [ ]:
ind = []
for i, res in enumerate(results['ids']):
  try:
    ind.append(res.index(f'id {i}'))
  except:
    ind.append(1000)

In [ ]:
ind

In [ ]:
sum(ind) / len(ind)

In [ ]:
df[:5].text.values

In [ ]:
questions = ['What are short-sellers seeing again?',
             'What is Carlyle Group known for?',
             'What factors are expected to hang over the stock market next week?',
             'Why have authorities halted oil export flows from the main pipeline in southern Iraq?',
             'What economic menace do world oil prices present?']

In [ ]:
q_embeds = embedder.get_embeddings(questions)

In [ ]:
results = collection_cos.query(q_embeds,5)

In [ ]:
results

# LAB6

### model1: "deepset/roberta-base-squad2"

In [ ]:
questions = ['What political party is currently in power in the United States?',
             'Who won the FIFA World Cup in 2018?',
             'What is the latest technological advancement in computer processors?',
             'Who is the 46th President of the United States?',
             'Which country hosted the 2020 Summer Olympics?',
             'What is the most popular social media platform worldwide?',
             'Which sport requires the use of a shuttlecock?',
             'What is the most-watched television series of all time?',
             'Who is the CEO of Tesla Inc.?',
             'Which country invented the internet?',
             'Who is the current Prime Minister of the United Kingdom?',
             'Which team has won the most Super Bowl titles?',
             'What is the latest version of the iPhone?',
             'Who is considered the greatest basketball player of all time?',
             'What is the currency of Japan?',
             'Who created the theory of relativity?',
             'Which country has the largest population in the world?',
             'What is the main component of solar panels?',
             'Who wrote the novel "1984"?',
             'Which country won the most medals in the 2021 Tokyo Olympics?',
             'What is the largest tech company in the world by revenue?',
             'Which city is hosting the 2022 FIFA World Cup?',
             'Who won the Nobel Peace Prize in 2020?',
             'What is the fastest land animal?',
             'Who is the richest person in the world?',
             'What is the capital of Australia?',
             'What is the worlds largest social media network?',
             'Who discovered the theory of evolution?',
             'Which country won the 2019 Rugby World Cup?',
             'What is the newest gaming console released by Sony?',
             'Who wrote the play "Romeo and Juliet"?']

answers = ['The Democratic Party',
           'France',
           'The introduction of 7-nanometer processors.',
           'Joe Biden',
           'Japan',
           'Facebook',
           'Badminton',
           'Game of Thrones',
           'Elon Musk',
           'United States',
           'Boris Johnson',
           'Pittsburgh Steelers',
           'iPhone 13',
           'Michael Jordan',
           'Japanese yen',
           'Albert Einstein',
           'China',
           'Silicon',
           'George Orwell',
           'United States',
           'Apple Inc.',
           'Qatar',
           'World Food Programme',
           'Cheetah',
           'Elon Musk',
           'Canberra',
           'Facebook',
           'Charles Darwin',
           'South Africa',
           'PlayStation 5',
           'William Shakespeare']

In [ ]:
q_embeds = embedder.get_embeddings(questions)

In [ ]:
results = collection_cos.query(q_embeds,5)

In [ ]:
results['documents'][0]

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [ ]:
!pip install evaluate
!pip install bert_score
from evaluate import load
bertscore = load("bertscore")

In [ ]:
bs_all = []
for q, a, index in zip(questions, answers, range(len(answers))):
  QA_input = {'question': q,
             'context': ' '.join(results['documents'][index])}
  res = nlp(QA_input)
  bs = bertscore.compute(predictions=[res['answer']], references=[a], lang="en")
  bs_all.append(bs)

  print(f'Question: {q}\nAnswer: {res["answer"]}\nUser answer: {a}\nScore: {bs["f1"][0]}\n ')

In [ ]:
f1_scores = [bs['f1'][0] for bs in bs_all]
sum(f1_scores)/len(f1_scores)

In [ ]:
questions = ['What are short-sellers seeing again?',
             'What is Carlyle Group known for?',
             'What factors are expected to hang over the stock market next week?',
             'Why have authorities halted oil export flows from the main pipeline in southern Iraq?',
             'What economic menace do world oil prices present?',
             'What offset the positive outlook from computer maker Dell Inc. on stocks?',
             'How much did the assets of the nation\'s retail money market mutual funds fall in the latest week?',
             'What indicators suggest that the economy is improving?',
             'What was Danny Bazil Riley\'s initial job and salary?',
             'What are short-sellers seeing again on Wall Street?',

             'Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums?',
             'Can OPEC do anything to douse oil prices in an oversupplied market?',
             'Non-OPEC oil exporters should consider increasing output to cool record crude prices, according to OPEC President Purnomo Yusgiantoro. What is the suggestion amidst soaring oil prices?',
             'Did the auction for Google Inc.\'s initial public offering start off smoothly?',
             'What happened to the dollar on Friday amidst data showing a record U.S. trade deficit?',
             'When should you have the money talk with your elderly relatives?',
             'Why has the back-to-school season become a huge marketing phenomenon?',
             'Who has lost less in the stock market compared to growth-focused investors?',
             'What has driven the US trade deficit to a record high?',
             'According to a press report, which company could be bracing itself for a possible takeover attempt?',

             'Were there last-minute worries over an interview with Google\'s bosses in Playboy magazine?',
             'According to official figures, is the eurozone economy still growing?',
             'What is causing the slowdown in economic growth in Japan?',
             'What was the impact of the South African central bank trimming interest rates without prior warning?',
             'According to a new survey, what happened to the cost of buying cars over the past five years?',
             'Why did South Korea\'s central bank cut interest rates?',
             'When did the auction of shares in Google take place?',
             'Why did Hewlett-Packard shares fall and what is their warning for the final quarter?',
             'What happened to one of the oldest textile operators on the Indian Ocean island of Mauritius?',
             'Why did Chad ask the IMF for a loan?'


]


answers = ['green',
           'making well-timed and occasionally controversial plays in the defense industry',
           'soaring crude prices, worries about the economy and the outlook for earnings',
           'intelligence showed a rebel militia could strike infrastructure',
           'new economic menace barely three months before the US presidential elections',
           'oil prices surged past #36;46 a barrel', '#36;1.17 billion',
           'retail sales bounced back and new claims for jobless benefits fell',
           'general manager at a commercial real estate firm at an annual base salary of #36;70,000',
           'green',

           'soaring crude prices plus worries about the economy and the outlook for earnings',
           'no, markets are already oversupplied by 2.8 million barrels per day (bpd) of crude',
           'increasing output to cool record crude prices',
           'no, it got off to a rocky start after the Web search company sidestepped a bullet from U.S. securities regulators',
           'it tumbled broadly',
           'soon',
           'the purchasing power of kids',
           'investors in value-focused mutual funds',
           'oil costs',
           'Shell',

           'Yes',
           'Yes, but there are warnings it may slow down later in the year',
           'drop in domestic and corporate spending',
           'the lack of warning hits the rand and surprises markets',
           'The cost of buying both new and second-hand cars fell sharply',
           'to drive growth in the economy',
           'on Friday',
           'disappointing third-quarter profits, final quarter will also fall short of expectations',
           'it shut seven factories and cut 900 jobs',
           'to pay for looking after more than 100,000 refugees from conflict-torn Darfur in western Sudan'
           ]

In [ ]:
q_embeds = embedder.get_embeddings(questions)

In [ ]:
results = collection_cos.query(q_embeds,5)

In [ ]:
bs_all = []
for q, a, index in zip(questions, answers, range(len(answers))):
  QA_input = {'question': q,
             'context': ' '.join(results['documents'][index])}
  res = nlp(QA_input)
  bs = bertscore.compute(predictions=[res['answer']], references=[a], lang="en")
  bs_all.append(bs)

  print(f'Question: {q}\nAnswer: {res["answer"]}\nUser answer: {a}\nScore: {bs["f1"][0]}\n ')

In [ ]:
f1_scores = [bs['f1'][0] for bs in bs_all]
sum(f1_scores)/len(f1_scores)

### model 2: distilbert-base-cased-distilled-squad

In [ ]:
model_name = 'distilbert-base-cased-distilled-squad'

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [ ]:
questions = ['What political party is currently in power in the United States?',
             'Who won the FIFA World Cup in 2018?',
             'What is the latest technological advancement in computer processors?',
             'Who is the 46th President of the United States?',
             'Which country hosted the 2020 Summer Olympics?',
             'What is the most popular social media platform worldwide?',
             'Which sport requires the use of a shuttlecock?',
             'What is the most-watched television series of all time?',
             'Who is the CEO of Tesla Inc.?',
             'Which country invented the internet?',
             'Who is the current Prime Minister of the United Kingdom?',
             'Which team has won the most Super Bowl titles?',
             'What is the latest version of the iPhone?',
             'Who is considered the greatest basketball player of all time?',
             'What is the currency of Japan?',
             'Who created the theory of relativity?',
             'Which country has the largest population in the world?',
             'What is the main component of solar panels?',
             'Who wrote the novel "1984"?',
             'Which country won the most medals in the 2021 Tokyo Olympics?',
             'What is the largest tech company in the world by revenue?',
             'Which city is hosting the 2022 FIFA World Cup?',
             'Who won the Nobel Peace Prize in 2020?',
             'What is the fastest land animal?',
             'Who is the richest person in the world?',
             'What is the capital of Australia?',
             'What is the worlds largest social media network?',
             'Who discovered the theory of evolution?',
             'Which country won the 2019 Rugby World Cup?',
             'What is the newest gaming console released by Sony?',
             'Who wrote the play "Romeo and Juliet"?']

answers = ['The Democratic Party',
           'France',
           'The introduction of 7-nanometer processors.',
           'Joe Biden',
           'Japan',
           'Facebook',
           'Badminton',
           'Game of Thrones',
           'Elon Musk',
           'United States',
           'Boris Johnson',
           'Pittsburgh Steelers',
           'iPhone 13',
           'Michael Jordan',
           'Japanese yen',
           'Albert Einstein',
           'China',
           'Silicon',
           'George Orwell',
           'United States',
           'Apple Inc.',
           'Qatar',
           'World Food Programme',
           'Cheetah',
           'Elon Musk',
           'Canberra',
           'Facebook',
           'Charles Darwin',
           'South Africa',
           'PlayStation 5',
           'William Shakespeare']

In [ ]:
q_embeds = embedder.get_embeddings(questions)

In [ ]:
results = collection_cos.query(q_embeds,5)

In [ ]:
bs_all = []
for q, a, index in zip(questions, answers, range(len(answers))):
  QA_input = {'question': q,
             'context': ' '.join(results['documents'][index])}
  res = nlp(QA_input)
  bs = bertscore.compute(predictions=[res['answer']], references=[a], lang="en")
  bs_all.append(bs)

  print(f'Question: {q}\nAnswer: {res["answer"]}\nUser answer: {a}\nScore: {bs["f1"][0]}\n ')

In [ ]:
f1_scores = [bs['f1'][0] for bs in bs_all]
sum(f1_scores)/len(f1_scores)

### Gradio

In [ ]:
!pip install gradio==3.48.0
import gradio as gr

In [ ]:
def echo(question, history):
    q_embeds = embedder.get_embeddings([question])
    results = collection_cos.query(q_embeds,5)
    QA_input = {'question': question,
             'context': ' '.join(results['documents'][0])}
    res = nlp(QA_input)
    return res['answer']

In [ ]:
demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="QA Bot")
demo.launch()